**Loading Libraries**

In [ ]:
#Loading Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [10]:
#Creating a dataframe
df=pd.read_csv('D:\Personnal\E-Commerce_Data\data.csv', encoding='latin1')

In [11]:
# Display basic information about the dataset
print("Dataset Info:")
print(df.info())
print("\nFirst 5 rows:")
print(df.head())
print("\nDataset shape:", df.shape)

Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB
None

First 5 rows:
  InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                  WHITE METAL LANTERN         6   
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3    536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4  

In [12]:
# Data Cleaning Steps

# 1. Check for missing values
print("\nMissing values before cleaning:")
print(df.isnull().sum())

# 2. Handle missing values
# Fill missing CustomerID with 0 (or you could use mean/median)
df['CustomerID'] = df['CustomerID'].fillna(0)

# For Description, fill with 'Unknown' if missing
df['Description'] = df['Description'].fillna('Unknown')

# 3. Check for duplicates
print(f"\nNumber of duplicate rows: {df.duplicated().sum()}")

# Remove duplicates if any
df = df.drop_duplicates()
print(f"Shape after removing duplicates: {df.shape}")

# 4. Data type conversions
# Convert InvoiceDate to datetime
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'], format='%m/%d/%Y %H:%M')

# Convert CustomerID to integer (after filling NaN)
df['CustomerID'] = df['CustomerID'].astype(int)

# 5. Handle negative quantities (returns/cancellations)
print(f"\nRows with negative quantities: {len(df[df['Quantity'] < 0])}")

# You might want to separate returns from regular sales
# For now, we'll keep them but you could filter them out:
# df = df[df['Quantity'] >= 0]

# 6. Check for outliers in numerical columns
numerical_cols = ['Quantity', 'UnitPrice']
print("\nSummary statistics for numerical columns:")
print(df[numerical_cols].describe())

# 7. Handle outliers (optional - based on business rules)
# For example, remove rows where Quantity is beyond reasonable limits
df = df[(df['Quantity'] >= -1000) & (df['Quantity'] <= 1000)]
df = df[df['UnitPrice'] >= 0]

# 8. Clean text columns - remove extra spaces
text_columns = ['Description', 'Country']
for col in text_columns:
    df[col] = df[col].str.strip()

# 9. Standardize country names (example)
df['Country'] = df['Country'].str.title()

# 10. Check for inconsistent StockCode values
print(f"\nUnique StockCode values: {df['StockCode'].nunique()}")

# 11. Create new useful columns
# Extract date components
df['Year'] = df['InvoiceDate'].dt.year
df['Month'] = df['InvoiceDate'].dt.month
df['Day'] = df['InvoiceDate'].dt.day
df['Hour'] = df['InvoiceDate'].dt.hour

# Calculate total price
df['TotalPrice'] = df['Quantity'] * df['UnitPrice']

# 12. Final data quality check
print("\nFinal dataset info:")
print(df.info())
print(f"\nFinal shape: {df.shape}")
print("\nMissing values after cleaning:")
print(df.isnull().sum())


Missing values before cleaning:
InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

Number of duplicate rows: 5268
Shape after removing duplicates: (536641, 8)

Rows with negative quantities: 10587

Summary statistics for numerical columns:
            Quantity      UnitPrice
count  536641.000000  536641.000000
mean        9.620029       4.632656
std       219.130156      97.233118
min    -80995.000000  -11062.060000
25%         1.000000       1.250000
50%         3.000000       2.080000
75%        10.000000       4.130000
max     80995.000000   38970.000000

Unique StockCode values: 4062

Final dataset info:
<class 'pandas.core.frame.DataFrame'>
Index: 536467 entries, 0 to 541908
Data columns (total 13 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    536467 non-null  obj

In [13]:
# Display sample of cleaned data
print("\nSample of cleaned data:")
print(df.head(10))


Sample of cleaned data:
  InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                  WHITE METAL LANTERN         6   
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3    536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4    536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   
5    536365     22752         SET 7 BABUSHKA NESTING BOXES         2   
6    536365     21730    GLASS STAR FROSTED T-LIGHT HOLDER         6   
7    536366     22633               HAND WARMER UNION JACK         6   
8    536366     22632            HAND WARMER RED POLKA DOT         6   
9    536367     84879        ASSORTED COLOUR BIRD ORNAMENT        32   

          InvoiceDate  UnitPrice  CustomerID         Country  Year  Month  \
0 2010-12-01 08:26:00       2.55       17850  United Kingdom  2010     12   
1 2010-12-01 08:26:00       

In [ ]:
import pyodbc

# Define connection parameters
server = 'KINUSHDK'        # e.g. 'localhost' or '192.168.1.10'
database = 'E_Commerce_Data'    # e.g. 'testdb'
username = 'sa'         # e.g. 'sa'
password = 'sekonda'         # e.g. 'mypassword'

# Create connection string
conn_str = (
    f'DRIVER={{ODBC Driver 17 for SQL Server}};'
    f'SERVER={server};'
    f'DATABASE={database};'
    f'UID={username};'
    f'PWD={password}'
)

# Connect to the database
conn = pyodbc.connect(conn_str)

# Create a cursor
cursor = conn.cursor()

# Run a query
cursor.execute("SELECT @@VERSION")

# Fetch results
row = cursor.fetchone()
print("SQL Server version:", row[0])

# Close connection
conn.close()